In [45]:

import sys
import csv
import math
import copy
import time
import numpy as np
from collections import Counter
from numpy import *
import pandas as pd

## Multiclass classifier decision tree using ID3 algorithm

In [46]:
feature = []

In [47]:
#normalize the entire dataset prior to learning using min-max normalization 
def normalize(matrix):
    a=np.array(matrix)
    a=a.astype(np.float)
    #print("Before normalizing")
    #print(a)
    b = np.apply_along_axis(lambda x: (x-np.min(x))/float(np.max(x)-np.min(x)),0,a)
    #print(b)
    return b
    

In [48]:
# reading from the file using numpy genfromtxt with delimiter ','
def load_csv(file):
    X = genfromtxt(file, delimiter=",",dtype=str)
    return X

In [49]:
#method to randomly shuffle the array using the numpy.random.shuffle()
def random_numpy_array(ar):
    np.random.shuffle(ar)
    arr = ar
    #print(arr)
    return arr

In [58]:
#Normalize the data and generate the training labels,training features, test labels and test training
def generate_set(X):
    #print(X.shape[0])
    Y = X[:,-1]
    j = Y.reshape(len(Y),1)
    #print("J is",j)
    
    new_X = X[:,:-1]
    #normalize the data step
    normalized_X = normalize(new_X)
    X = np.concatenate((normalized_X,j),axis=1)
    
    rows_size = X.shape[0]
    num_test = round(0.1*rows_size)
    
    start = 0
    end = num_test
    
    test_features =[]
    test_labels =[]
    train_features = []
    train_labels = []
    
    for i in range(10):
        X_test = X[start:end , :]
        before = X[:start, :]
        after = X[end:, :]
        X_train = np.concatenate((before, after), axis=0)

        #print("Before normalizing",X_test)
        y_test = X_test[:, -1].flatten()
        y_train = X_train[:,-1].flatten()

        X_test = X_test[:,:-1]
        X_train = X_train[:,:-1]
        X_test = X_test.astype(np.float)
        X_train = X_train.astype(np.float)
        test_features.append(X_test)
        test_labels.append(y_test)
        train_features.append(X_train)
        train_labels.append(y_train)
        #print("start: ",start)
        #print("end: ",end)
        start = end
        end = end+num_test
    return test_features,test_labels,train_features,train_labels

In [59]:
#build a dictionary where the key is the class label and values are the features which belong to that class.
def build_dict_of_attributes_with_class_values(X,y):
    
    dict_of_attri = {}
    fea_list =[]
    
    for i in range(X.shape[1]):
        fea_index = i
        all_values = X[:,i]
        #print("all_values: "all_values)
        attribute_list =[]

        for index, j in enumerate(all_values):
            attribute_value = []
            attribute_value.append(j)
            attribute_value.append(y[index])
            attribute_list.append(attribute_value)

        dict_of_attri[fea_index]= attribute_list
        fea_list.append(fea_index)
    
    return dict_of_attri,fea_list


In [60]:
# Iterative Dichotomiser 3 entropy calculation
def entropy(y):
    class_freq = {}
    attri_entropy = 0.0
    
    for i in y:
        if i in class_freq:
            class_freq[i] += 1
        else:
            class_freq[i] = 1
    #print(class_freq)
    
    for freq in class_freq.values():
        attri_entropy += (-freq/float(len(y))) * math.log(freq/float(len(y)),2)
    #print(attri_entropy)
    
    return attri_entropy


In [61]:
#Class node and explanation is self explaination
class Node(object):
#     init the node with val,lchild,rchild,thea and leaf.
    def __init__(self, val, lchild, rchild,thea,leaf):
        self.root_value = val
        self.root_left = lchild
        self.root_right = rchild
        self.theta = thea
        self.leaf = leaf

#     method to identify if the node is leaf
    def is_leaf(self):
        
        return self.leat

#     method to return threshold value
    def ret_thetha(self):
        
        return self.theta
    
#     method return root value
    def ret_root_value(self):
        
        return root_value
    
#     method return left tree
    def ret_llist(self):
        
        return root_left

#     method return right tree
    def ret_rlist(self):
        
        return root_right

    def __repr__(self):
        return "(%r, %r, %r, %r)" %(self.root_value,self.root_left,self.root_right,self.theta)


In [70]:
#Decision tree object
class DecisionTree(object):
    fea_list = []
    def __init__(self):
        self.root_node = None

    #method to return the major class value
    def cal_major_class_values(self,class_values):
        data = Counter(class_values).most_common(1)[0][0]
        return data

    #method to calculate best threshold value for each feature
    def cal_best_theta_value(self,ke,attri_list):
        data = []
        class_labels = []
        
        for i in attri_list:
            data.append(i[0])
            class_labels.append(i[1])
            
        feature_values_entropy = entropy(class_labels)

        max_info_gain = 0
        theta = 0
        
        best__left_index = []
        best_right_index = []
        after_split = []
        
        data.sort()
        
        for i in range(len(data) - 1):
            cur_theta = float(data[i]+data[i+1])/ 2
            index_less_theta = []
            values_less_theta = []
            index_greater_theta = []
            values_greater_theta = []

            for c,j in enumerate(attri_list):
                #print(c,j[0])
                if j[0] <= cur_theta:
                    values_less_theta.append(j[1])
                    index_less_theta.append(c)
                else:
                    values_greater_theta.append(j[1])
                    index_greater_theta.append(c)

            entropy_of_less = entropy(values_less_theta)
            #print(entropy_of_less)
            entropy_of_greater = entropy(values_greater_theta)
            #print(entropy_of_greater)

            info_gain = feature_values_entropy - (entropy_of_less*(len(index_less_theta)/(len(data)))) \
                            - (entropy_of_greater*(len(index_greater_theta)/(len(data))))

            if info_gain > max_info_gain:
                max_info_gain = info_gain
                theta = cur_theta
                best_left_index = index_less_theta
                best_right_index = index_greater_theta
                after_split = values_less_theta + values_greater_theta
        
        return max_info_gain, theta, best_left_index, best_right_index, after_split

    #method to select the best feature out of all the features.
    def best_feature(self,dict_rep):
        
        key_value = None
        best_info_gain = -1
        best_theta = 0
        best_left = []
        best_right = []
        best_class = []
        result_list = []
        for ke in dict_rep.keys():
            #print("Key ",ke)
            info_gain, theta, index_left, index_right, after_split = self.cal_best_theta_value(ke,dict_rep[ke])
            #print("Best theta ", ke, info_gain, theta, index_left)
            if info_gain > best_info_gain:
                best_info_gain = info_gain
                best_theta = theta
                key_value = ke
                best_left = index_left
                best_right = index_right
                best_class = after_split
        
        result_list.append(key_value)
        result_list.append(best_theta)
        result_list.append(best_left)
        result_list.append(best_right)
        result_list.append(best_class)
        
        return result_list

    def get_remainder_dict(self,dict_of_everything,index_split):
        global fea_list
        split_dict = {}

        for ke in dict_of_everything.keys():
            val_list = []
            modified_list = []
            cor_val = dict_of_everything[ke]
            
            for index,val in enumerate(cor_val):
                
                if index not in index_split:
                    modified_list.append(val)
                    val_list.append(val[1])
            #print(modified_list)
            
            split_dict[ke] = modified_list

        return split_dict,val_list

    #method to create decision tree
    def create_decision_tree(self, dict_of_everything,class_val,eta_min_val):
        global fea_list

        #if all the class labels are same, then we are set
        if len(set(class_val)) ==1:
            root_node =  Node(class_val[0],None,None,0,True)
            return root_node
        
        #if the no class vales are less than threshold, we assign the class with max values as the class label    
        elif len(class_val) < eta_min_val:
            majority_val = self.cal_major_class_values(class_val)
            root_node = Node(majority_val,None,None,0,True)
            return root_node

        else:
            best_features_list = self.best_feature(dict_of_everything)
            #print(best_features_list)
            node_name = best_features_list[0]
            theta = best_features_list[1]
            left_split = best_features_list[2]
            right_split = best_features_list[3]
            class_labels = best_features_list[4]
            
            left_tree,left_val = self.get_remainder_dict(dict_of_everything,left_split)
            right_tree,right_val = self.get_remainder_dict(dict_of_everything,right_split)
            
            left_child = self.create_decision_tree(left_tree,left_val,eta_min_val)
            right_child = self.create_decision_tree(right_tree,right_val,eta_min_val)
            
            root_node = Node(node_name,right_child,left_child,theta,False)
            
            return root_node

            

    def fit(self, dict_of_everything,cl_val,features,eta_min_val):
        global fea_list
        fea_list = features
        root_node = self.create_decision_tree(dict_of_everything,cl_val,eta_min_val)
        
        return root_node

    def classify(self,row,root):
        test_dict ={}
        
        for k,j in enumerate(row):
            test_dict[k] = j
        #print(test_dict)
        
        current_node = root
        
        while not current_node.leaf:
            #print(current_node.root_value,test_dict[current_node.root_value], current_node.theta)
            if test_dict[current_node.root_value] <= current_node.theta:
                current_node = current_node.root_left
            else:
                current_node = current_node.root_right
        #print(current_node.root_value,test_dict[current_node.root_value], current_node.theta)
        return current_node.root_value
        
    #method to the labels for the test data
    def predict(self, X, root):
        predict_list = []
        for row in X:
            y_pred = self.classify(row,root)
            predict_list.append(y_pred)
        return predict_list

In [71]:
#calculating the predicited accuracy
def accuracy_for_predicted_values(test_class_names1,l):
    true_ctr = 0
    false_ctr = 0
    
    for i in range(len(test_class_names1)):
        if(test_class_names1[i] == l[i]):
            true_ctr += 1
        else:
            false_ctr += 1
    
    return true_ctr, false_ctr, float(true_ctr)/len(l)

In [83]:
def main(num_arr, eta_min):
    eta_min_val = round(eta_min*num_arr.shape[0])
    sh_arr = random_numpy_array(num_arr)
    test_features, test_labels, train_features, train_labels = generate_set(sh_arr)
    cumulate_acc = 0
    
    standard_dev_set = []
    sum_of_squares = 0.0
    
    #ten fold iteration 
    for i in range(10):
        dict_labels,fea = build_dict_of_attributes_with_class_values(train_features[i], train_labels[i])
        decision_tree = DecisionTree()
        decision_tree_model = decision_tree.fit(dict_labels,train_labels[i],fea,eta_min_val)
        pred_labels = decision_tree.predict(test_features[i], decision_tree_model)
    
        right,wrong,accu = accuracy_for_predicted_values(test_labels[i], pred_labels)

        cumulate_acc += accu
        
        standard_dev_set.append(accu)      
        standard_mean = np.mean(standard_dev_set)
        
        for i in standard_dev_set:
            sum_of_squares += (i-standard_mean)**2
        
        final_deviation = np.sqrt(sum_of_squares/len(standard_dev_set))
        
        
        print("Accuracy is ",accu)
    print("Accuracy across 10-cross validation for",eta_min,"is",float(cumulate_acc)/10)
    print("Standard deviation is ", final_deviation)


In [84]:
eta_min_list = [0.05,0.10,0.15,0.20]
newfile = "iris.csv"
num_arr = load_csv(newfile)
for i in eta_min_list:
    main(num_arr,i)

Accuracy is  1.0
Accuracy is  0.9333333333333333
Accuracy is  0.8666666666666667
Accuracy is  0.9333333333333333
Accuracy is  0.8
Accuracy is  1.0
Accuracy is  1.0
Accuracy is  0.9333333333333333
Accuracy is  0.8666666666666667
Accuracy is  0.9333333333333333
Accuracy across 10-cross validation for 0.05 is 0.9266666666666667
Standard deviation is  0.14955076763014433
Accuracy is  0.8666666666666667
Accuracy is  0.9333333333333333
Accuracy is  1.0
Accuracy is  0.9333333333333333
Accuracy is  1.0
Accuracy is  0.8666666666666667
Accuracy is  1.0
Accuracy is  0.9333333333333333
Accuracy is  1.0
Accuracy is  0.9333333333333333
Accuracy across 10-cross validation for 0.1 is 0.9466666666666667
Standard deviation is  0.11960620276129942
Accuracy is  1.0
Accuracy is  1.0
Accuracy is  1.0
Accuracy is  1.0
Accuracy is  0.8
Accuracy is  0.9333333333333333
Accuracy is  0.9333333333333333
Accuracy is  1.0
Accuracy is  0.9333333333333333
Accuracy is  0.9333333333333333
Accuracy across 10-cross valida

In [74]:
eta_min_list = [0.05,0.10,0.15,0.20,0.25]
newfile = "sambase.csv"
num_arr = load_csv(newfile)
for i in eta_min_list:
    main(num_arr,i)

OSError: sambase.csv not found.